<a href="https://colab.research.google.com/github/liamshatzel/fcc-ml/blob/main/sms_text_classification_Liam_Shatzel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [89]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
  !pip install tensorflow
  !pip install keras
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import EarlyStopping
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
print(tf.__version__)

2.5.0


In [90]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2021-08-01 02:40:51--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.05s   

2021-08-01 02:40:51 (6.72 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2021-08-01 02:40:51--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[====

In [91]:
#preprocessing
train_data = pd.read_csv("train-data.tsv", encoding = "latin-1", sep = '\t')
train_data = pd.DataFrame(train_data)
train_data = train_data.rename(columns={"ham":"label", "ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.":"sms"})


test_data = pd.read_csv("valid-data.tsv", encoding = "latin-1", sep = '\t')
test_data = pd.DataFrame(test_data)
test_data = test_data.rename(columns={"ham":"label", "i am in hospital da. . i will return home in evening":"sms"})


In [92]:
print(len(train_data))
train_data.head()
train_data.label.value_counts()
train_data.describe()

4178


,label,sms
count,4178,4178
unique,2,3934
top,ham,"sorry, i'll call later"
freq,3618,22


In [93]:
train_data.loc[:,'label'] = train_data.label.map({'ham':0, 'spam':1})
print(train_data.shape)
train_data.head()

(4178, 2)


,label,sms
0,0,you can never do nothing
1,0,"now u sound like manky scouse boy steve,like! ..."
2,0,mum say we wan to go then go... then she can s...
3,0,never y lei... i v lazy... got wat? dat day Ã¼...
4,0,in xam hall boy asked girl tell me the startin...


In [94]:
test_data.loc[:,'label'] = test_data.label.map({'ham':0, 'spam':1})
print(test_data.shape)
test_data.head()

(1391, 2)


,label,sms
0,0,"not much, just some textin'. how bout you?"
1,0,i probably won't eat at all today. i think i'm...
2,0,donât give a flying monkeys wot they think a...
3,0,who are you seeing?
4,0,your opinion about me? 1. over 2. jada 3. kusr...


In [95]:
x_train, y_train = (train_data['sms'].values, train_data['label'].values)
count_vector = CountVectorizer()
#training_data = count_vector.fit_transform(x_train)

x_test, y_test = (test_data['sms'].values, test_data['label'].values)
#testing_data = count_vector.fit_transform(x_test)
'''
train_array = count_vector.transform(x_train).toarray()
train_array
frequency_matrix = pd.DataFrame(train_array, columns = count_vector.get_feature_names())
frequency_matrix'''


'\ntrain_array = count_vector.transform(x_train).toarray()\ntrain_array\nfrequency_matrix = pd.DataFrame(train_array, columns = count_vector.get_feature_names())\nfrequency_matrix'

In [96]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [97]:
token = Tokenizer()
token.fit_on_texts(x_train)

In [99]:
encoded_train = token.texts_to_sequences(x_train)
encoded_test = token.texts_to_sequences(x_test)
print(encoded_train[0:2])

[[3, 29, 281, 27, 340], [19, 6, 830, 55, 3665, 3666, 357, 3667, 55, 1, 9, 3668, 18, 94, 414, 85, 493, 113, 6, 3669, 41, 3670, 434, 537]]


In [100]:
max_length = 8

padded_train = pad_sequences(encoded_train, maxlen = max_length, padding = 'post')
padded_test = pad_sequences(encoded_test, maxlen = max_length, padding = 'post')

print(padded_train)

[[   3   29  281 ...    0    0    0]
 [ 493  113    6 ... 3670  434  537]
 [  88   29 3671 ...   94 3673 3674]
 ...
 [  19  272  162 ... 3591  220  191]
 [   6 3664   87 ...  749    2 1113]
 [ 248   12   37 ... 7691   57  179]]


In [101]:
vocab_size = len(token.word_index) + 1

In [102]:
'''model = Sequential()
model.add(Embedding(vocab_size, 24, input_length = max_length))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(1))'''

In [219]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [223]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [221]:
#model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
#print(model.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, None, 64)          492288    
_________________________________________________________________
bidirectional_14 (Bidirectio (None, None, 128)         66048     
_________________________________________________________________
bidirectional_15 (Bidirectio (None, 64)                41216     
_________________________________________________________________
dense_30 (Dense)             (None, 128)               8320      
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_12 (Dropout)         (None, 64)              

In [224]:
print(y_test)

[0 0 0 ... 0 1 1]


In [225]:
callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
model.fit(x=padded_train, y=y_train, epochs=50, validation_data = (padded_test, y_test), callbacks= [callback])

Epoch 1/50
131/131 [==============================] - 18s 51ms/step - loss: 0.5881 - accuracy: 0.8660 - val_loss: 0.3816 - val_accuracy: 0.8656
Epoch 2/50
131/131 [==============================] - 3s 23ms/step - loss: 0.3004 - accuracy: 0.8660 - val_loss: 0.2039 - val_accuracy: 0.8656
Epoch 3/50
131/131 [==============================] - 3s 23ms/step - loss: 0.1628 - accuracy: 0.8787 - val_loss: 0.1300 - val_accuracy: 0.9446
Epoch 4/50
131/131 [==============================] - 3s 23ms/step - loss: 0.0780 - accuracy: 0.9773 - val_loss: 0.0833 - val_accuracy: 0.9806
Epoch 5/50
131/131 [==============================] - 3s 25ms/step - loss: 0.0390 - accuracy: 0.9916 - val_loss: 0.0880 - val_accuracy: 0.9820
Epoch 6/50
131/131 [==============================] - 3s 25ms/step - loss: 0.0198 - accuracy: 0.9950 - val_loss: 0.0950 - val_accuracy: 0.9792
Epoch 7/50
131/131 [==============================] - 3s 25ms/step - loss: 0.0145 - accuracy: 0.9990 - val_loss: 0.1062 - val_accuracy: 0.979

In [226]:
from sklearn.metrics import confusion_matrix, accuracy_score

preds = (model.predict(padded_test) > 0.5).astype("int32")
accuracy_score(y_test, preds)

0.9755571531272466

In [227]:
pred_text = ["how are you doing today?"]
prediction = []
sms = pred_text
sms_pred = token.texts_to_sequences(sms)
sms_pred = pad_sequences(sms_pred, maxlen=max_length, padding='post')
print(model.predict(sms_pred))
pred = (model.predict(sms_pred) > 0.5).astype("int32").flatten().item()
print(pred)
'''if pred == 1:
  prediction.append('spam')
else:
  prediction.append('ham')'''

[[-19.514526]]
0


"if pred == 1:\n  prediction.append('spam')\nelse:\n  prediction.append('ham')"

In [235]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_text = [pred_text]
  prediction = []
  sms = pred_text
  sms_pred = token.texts_to_sequences(sms)
  sms_pred = pad_sequences(sms_pred, maxlen=max_length, padding='post')
  pred = (model.predict(sms_pred) >= -13).astype("int32").item()
  prediction.append(model.predict(sms_pred).item())
  if pred == 0:
    prediction.append('ham')
  else:
    prediction.append('spam')
  return (prediction)



In [236]:
pred_text = "our new mobile video service is live. just install on your phone to start watching."
                  
prediction = predict_message(pred_text)
print(prediction)

[-12.600882530212402, 'spam']


In [237]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
